# Project 2 - creating neural net from scratch

The objective of this project is to design a mini “deep learning framework” using only pytorch’s tensor operations and the standard math library, hence in particular without using autograd or the neural-network modules.

Your framework should import only `torch.FloatTensor` and `torch.LongTensor` from pytorch, and use no pre-existing neural-network python toolbox.

In [1]:
# Definitely allowed imports
import math
import torch
from torch import FloatTensor, LongTensor, Tensor
# additional imports


# our own written code

## Generate datasets

Generate a training and a test set of 1,000 points sampled uniformly in [0,1]2, each with a
label 0 if outside the disk of radius 1/ 2π and 1 inside,

In [143]:
def generate_disc_data(n=1000):
    inputs = torch.rand(n,2)
    euclidian_distances = torch.norm((inputs - torch.Tensor([[0.5, 0.5]])).abs(), 2, 1, True)
    targets = euclidian_distances.mul(math.sqrt(2*math.pi)).sub(1).sign().sub(1).div(2).abs().long()
    return inputs, targets

In [144]:
inputs, targets = generate_disc_data()
inputs, targets


 0.4061
 0.3824
 0.4093
   ⋮    
 0.4705
 0.3898
 0.0445
[torch.FloatTensor of size 1000x1]



(
  0.7600  0.1881
  0.6077  0.1331
  0.7716  0.1938
        ⋮        
  0.4142  0.9626
  0.1142  0.4441
  0.5361  0.4739
 [torch.FloatTensor of size 1000x2], 
     0
     1
     0
   ⋮   
     0
     1
     1
 [torch.LongTensor of size 1000x1])